In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import math

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import sympy as sphdd
def descenso_por_gradiente(w,b,i):
   # función a optimizar:
   f = ((tanh(w*i + b)+1)/2)**2
   
   gr
[] = etneid
   # optimizar los parametros de la funcion f

def gradiente_descendente(x_0):
   # Valor inicial de x
   x_t   = x_0
   alpha = 0.1
   tolerancia = 0.0001
   x_tsig = 1e16
   iter = 0
   while iter <= MAX_ITER:
      if abs(f(x_tsig) - f(x_t)) < tolerancia:
          break
      x_tsig = x_t - alpha * df(x_t)
      x_t = x_tsig
      iter = iter + 1

    return x_tsig


   # Para la optimizacion: arrancar de un punto cualquiera y seguir el gradiente en direccion opuesta
      # x es un punto inicial aleatorio
      # d es una lista de diagnosticos? o sea de 1sy 0s?
      # regla de actualizacion -> (x1, ..., xn) - alpha * gradiente((x1, ..., xn))
      # Criterio de convergencia (identifica un "plateau")
      
      

In [26]:
def abrirImagenesEscaladas( carpeta, escala=32 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [ ]:
def balancear_datos(imagenes_entrenamiento):
    imagenes_entrenamiento_balanceadas = 
    return imagenes_entrenamiento_balanceadas